<a href="https://colab.research.google.com/github/ereinha/SineKAN/blob/main/B_SplineKAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt

In [3]:
from kan import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchsummary import summary

from sklearn.metrics import precision_score, recall_score, f1_score

In [4]:
# Load MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
train_set = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
val_set = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
train_loader = DataLoader(train_set, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, num_workers=2, shuffle=False)

In [ ]:
epochs = 30
lrs = [4e-3, 5e-3, 6e-3]
gammas = [0.8, 0.9]
hdims = [64, 128, 256]
best_accs = []
for lr in lrs:
    for gamma in gammas:
        for hdim in hdims:
            torch.manual_seed(42)
            best_acc = 0
            # Define model
            model = KAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            # Define optimizer
            optimizer = optim.AdamW(model.parameters(), lr=lr)
            # Define learning rate scheduler
            scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

            # Define loss
            criterion = nn.CrossEntropyLoss()
            for epoch in range(epochs):
                # Train
                model.train()
                with tqdm(train_loader) as pbar:
                    for i, (images, labels) in enumerate(pbar):
                        images = images.view(-1, 28 * 28).to(device)
                        optimizer.zero_grad()
                        output = model(images)
                        loss = criterion(output, labels.to(device))
                        loss.backward()
                        optimizer.step()
                        accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                        pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

                # Validation
                model.eval()
                val_loss = 0
                val_accuracy = 0
                with torch.no_grad():
                    for images, labels in val_loader:
                        images = images.view(-1, 28 * 28).to(device)
                        output = model(images)
                        val_loss += criterion(output, labels.to(device)).item()
                        val_accuracy += (
                            (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                        )
                val_loss /= len(val_loader)
                val_accuracy /= len(val_loader)
                if val_accuracy > best_acc:
                    best_acc = val_accuracy

                # Update learning rate
                scheduler.step()

                print(
                    f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
                )
            best_accs.append(best_acc)
            print(f"LR: {lr} Gamma: {gamma} Hdim: {hdim} Best Accuracy: {best_acc}")

100%|██████████| 938/938 [00:23<00:00, 40.31it/s, accuracy=0.938, loss=0.268, lr=0.004]


Epoch 1, Val Loss: 0.12268612892693205, Val Accuracy: 0.9618829617834395


100%|██████████| 938/938 [00:21<00:00, 43.48it/s, accuracy=0.969, loss=0.0879, lr=0.0032]


Epoch 2, Val Loss: 0.10790182129186311, Val Accuracy: 0.9653662420382165


100%|██████████| 938/938 [00:22<00:00, 41.38it/s, accuracy=0.969, loss=0.1, lr=0.00256]


...

Epoch 28, Val Loss: 0.11702385954455671, Val Accuracy: 0.9810907643312102


100%|██████████| 938/938 [00:25<00:00, 36.96it/s, accuracy=1, loss=5.72e-5, lr=0.000314]


Epoch 29, Val Loss: 0.11788478989410252, Val Accuracy: 0.9811902866242038


100%|██████████| 938/938 [00:23<00:00, 39.73it/s, accuracy=1, loss=3.63e-6, lr=0.000283]


Epoch 30, Val Loss: 0.11804310040731328, Val Accuracy: 0.9812898089171974
LR: 0.006 Gamma: 0.9 Hdim: 256 Best Accuracy: 0.9812898089171974


In [ ]:
epochs = 30
wds = [0.01, 0.1, 0.25, 0.5, 0.75, 1.]
wd_accs = np.empty((6, epochs))
for h, wd in enumerate(wds):
    torch.manual_seed(42)
    best_acc = 0
    # Define model
    model = KAN(layers_hidden=[28 * 28, 64, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=wd)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )

        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        wd_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:20<00:00, 45.21it/s, accuracy=0.938, loss=0.205, lr=0.005]


Epoch 1, Val Loss: 0.14388836470023272, Val Accuracy: 0.9560111464968153


100%|██████████| 938/938 [00:19<00:00, 48.91it/s, accuracy=0.969, loss=0.0656, lr=0.0045]


Epoch 2, Val Loss: 0.13029202954416538, Val Accuracy: 0.9609872611464968


100%|██████████| 938/938 [00:20<00:00, 46.61it/s, accuracy=0.938, loss=0.165, lr=0.00405]


...

Epoch 28, Val Loss: 0.14144248596042586, Val Accuracy: 0.9653662420382165


100%|██████████| 938/938 [00:24<00:00, 38.73it/s, accuracy=1, loss=0.138, lr=0.000262]


Epoch 29, Val Loss: 0.14152032512412138, Val Accuracy: 0.9657643312101911


100%|██████████| 938/938 [00:22<00:00, 40.96it/s, accuracy=1, loss=0.0805, lr=0.000236]


Epoch 30, Val Loss: 0.14114037508465302, Val Accuracy: 0.9665605095541401


In [ ]:
epochs = 30
hdims = [16, 32, 64, 128, 256, 512]
hdim_metrics = {
    'accuracy': np.empty((6, epochs)),
    'precision': np.empty((6, epochs)),
    'recall': np.empty((6, epochs)),
    'f1_score': np.empty((6, epochs)),
}
for h, hdim in enumerate(hdims):
    torch.manual_seed(42)
    # Define model
    model = KAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=.01)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                preds = output.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        val_precision = precision_score(all_labels, all_preds, average='weighted')
        val_recall = recall_score(all_labels, all_preds, average='weighted')
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        hdim_metrics['accuracy'][h, epoch] = val_accuracy
        hdim_metrics['precision'][h, epoch] = val_precision
        hdim_metrics['recall'][h, epoch] = val_recall
        hdim_metrics['f1_score'][h, epoch] = val_f1

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, "
            f"Val Accuracy: {val_accuracy}, Precision: {val_precision}, "
            f"Recall: {val_recall}, F1 Score: {val_f1}"
        )

100%|██████████| 938/938 [00:23<00:00, 40.55it/s, accuracy=0.875, loss=0.318, lr=0.005]


Epoch 1, Val Loss: 0.21099451413247616, Val Accuracy: 0.9387, Precision: 0.9397798317873086, Recall: 0.9387, F1 Score: 0.938746202728289


100%|██████████| 938/938 [00:21<00:00, 43.69it/s, accuracy=0.938, loss=0.254, lr=0.0045]


Epoch 2, Val Loss: 0.18129041401527016, Val Accuracy: 0.9423, Precision: 0.9428686653245404, Recall: 0.9423, F1 Score: 0.9422563681420061


100%|██████████| 938/938 [00:22<00:00, 40.99it/s, accuracy=0.969, loss=0.0552, lr=0.00405]


...

Epoch 28, Val Loss: 0.09864768001959377, Val Accuracy: 0.9814, Precision: 0.9814150191737067, Recall: 0.9814, F1 Score: 0.9813958225846251


100%|██████████| 938/938 [00:25<00:00, 37.12it/s, accuracy=1, loss=2.49e-6, lr=0.000262]


Epoch 29, Val Loss: 0.09916071078565308, Val Accuracy: 0.9822, Precision: 0.9822062225126204, Recall: 0.9822, F1 Score: 0.9821950557695962


100%|██████████| 938/938 [00:23<00:00, 39.10it/s, accuracy=1, loss=9.69e-8, lr=0.000236]


Epoch 30, Val Loss: 0.10065157175277045, Val Accuracy: 0.9818, Precision: 0.9818135327211661, Recall: 0.9818, F1 Score: 0.9817948766708329


In [ ]:
np.save('./saved_results/bspline_hdim_metrics.npy', hdim_metrics)

In [ ]:
epochs = 30
n_layers = [1, 2, 3, 4]
nlayer_metrics = {
    'accuracy': np.empty((4, epochs)),
    'precision': np.empty((4, epochs)),
    'recall': np.empty((4, epochs)),
    'f1_score': np.empty((4, epochs)),
}
for h, n_layer in enumerate(n_layers):
    torch.manual_seed(42)
    # Define model
    model = KAN(layers_hidden=[28 * 28] + [128]*n_layer + [10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=.01)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                preds = output.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        val_precision = precision_score(all_labels, all_preds, average='weighted')
        val_recall = recall_score(all_labels, all_preds, average='weighted')
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        nlayer_metrics['accuracy'][h, epoch] = val_accuracy
        nlayer_metrics['precision'][h, epoch] = val_precision
        nlayer_metrics['recall'][h, epoch] = val_recall
        nlayer_metrics['f1_score'][h, epoch] = val_f1

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:22<00:00, 41.16it/s, accuracy=0.938, loss=0.247, lr=0.005]


Epoch 1, Val Loss: 0.16121391121929832, Val Accuracy: 0.9515


100%|██████████| 938/938 [00:21<00:00, 43.39it/s, accuracy=1, loss=0.0498, lr=0.0045]


Epoch 2, Val Loss: 0.15240780582687088, Val Accuracy: 0.9557


100%|██████████| 938/938 [00:23<00:00, 40.69it/s, accuracy=0.969, loss=0.0606, lr=0.00405]


...

Epoch 28, Val Loss: 0.17969782951720623, Val Accuracy: 0.9779


100%|██████████| 938/938 [00:29<00:00, 31.38it/s, accuracy=1, loss=1.62e-6, lr=0.000262]


Epoch 29, Val Loss: 0.1834418538215369, Val Accuracy: 0.9774


100%|██████████| 938/938 [00:30<00:00, 30.85it/s, accuracy=1, loss=5.66e-7, lr=0.000236]


Epoch 30, Val Loss: 0.19204274022891754, Val Accuracy: 0.9779


In [ ]:
np.save('./saved_results/bspline_layer_metrics.npy', nlayer_metrics)

In [5]:
from torch.profiler import profile, record_function, ProfilerActivity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def profile_model(model, inputs, num_runs=100):
    total_times = []
    for _ in range(num_runs):
        total_time = 0
        with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
            model(inputs)

        # Extract total CUDA time spent on 'model_inference'
        for event in prof.key_averages():
            total_time += event.device_time

        total_times.append(total_time)
    average_time = np.mean(total_times)
    std_time = np.std(total_times)
    print(f"Average inference time: {average_time} microseconds +/- {std_time}")
    return average_time, std_time

In [7]:
inputs = torch.randn((1, 784)).to(device)
hdims = [16, 32, 64, 128, 256, 512]
means = []
stds = []
for h in hdims:
    model = KAN(layers_hidden=[28 * 28, h, 10], grid_size=8).to(device)
    print('Hidden Dims: %d' % h)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)

Hidden Dims: 16
Average inference time: 66.47211507738096 microseconds +/- 0.11704291342026212
Hidden Dims: 32
Average inference time: 72.42283911309525 microseconds +/- 0.12245335748447216
Hidden Dims: 64
Average inference time: 67.28495214880952 microseconds +/- 0.10939488439911335
Hidden Dims: 128
Average inference time: 71.33039539880951 microseconds +/- 0.15683145651011335
Hidden Dims: 256
Average inference time: 83.39252026190475 microseconds +/- 0.32448593816482846
Hidden Dims: 512
Average inference time: 114.78516217857144 microseconds +/- 0.46170528652459214


In [8]:
np.save('./saved_results/bspline_scaling_mean_hdims.npy', means)
np.save('./saved_results/bspline_scaling_std_hdims.npy', stds)

In [9]:
hdims = [16, 32, 64, 128, 256, 512]
means = []
stds = []
for h in hdims:
    model = KAN(layers_hidden=[28 * 28, 64, 10], grid_size=8).to(device)
    inputs = torch.randn((h, 784)).to(device)
    print('Input Batch Size: %d' % h)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)

Input Batch Size: 16
Average inference time: 113.06373610714287 microseconds +/- 0.3852416861387589
Input Batch Size: 32
Average inference time: 128.35111681547616 microseconds +/- 0.1772670825445702
Input Batch Size: 64
Average inference time: 173.80980667261903 microseconds +/- 0.4350570564654399
Input Batch Size: 128
Average inference time: 313.99014177380957 microseconds +/- 0.3971190834169694
Input Batch Size: 256
Average inference time: 547.166382422619 microseconds +/- 1.0342619850670636
Input Batch Size: 512
Average inference time: 1008.308518077381 microseconds +/- 4.626423570715507


In [10]:
np.save('./saved_results/bspline_scaling_mean_batch.npy', means)
np.save('./saved_results/bspline_scaling_std_batch.npy', stds)

In [13]:
nlayers = [1, 2, 3, 4]
means = []
stds = []
for n in nlayers:
    model = KAN(layers_hidden=[28 * 28] + [64]*n +[10], grid_size=8).to(device)
    inputs = torch.randn((1, 784)).to(device)
    print('Number of Hidden Layers: %d' % n)
    mean, std = profile_model(model, inputs, 1000)
    means.append(mean)
    stds.append(std)

Number of Hidden Layers: 1
Average inference time: 67.25756924999999 microseconds +/- 0.10681720475688469
Number of Hidden Layers: 2
Average inference time: 68.16116807539683 microseconds +/- 0.08407886990816509
Number of Hidden Layers: 3
Average inference time: 66.09342230357143 microseconds +/- 0.08218290247522583
Number of Hidden Layers: 4
Average inference time: 65.05110593095239 microseconds +/- 0.06079233458194082


In [16]:
np.save('./saved_results/bspline_scaling_mean_layers.npy', means)
np.save('./saved_results/bspline_scaling_std_layers.npy', stds)